# Sale Forecasting using Agent

In [ ]:
# Importing Libraries
from langchain.chat_models import ChatOpenAI
import sqlite3
import pandas as pd
from dotenv import load_dotenv
import os
import openai

In [2]:
load_dotenv()
OpenAI_API_KEY = os.getenv("OpenAI_API_KEY")

### Initializing Langchain LLM models

In [ ]:
model = ChatOpenAI(openai_api_key=OpenAI_API_KEY,model_name="gpt-4o-mini")

In [4]:
import chardet

# Detect the encoding
with open(r"C:\Users\eDominer\Python Project\Sales Prediction\sales_data_sample.csv", 'rb') as f:
    result = chardet.detect(f.read())

# Use the detected encoding to read the file
encoding = result['encoding']

In [5]:
df = pd.read_csv('sales_data_sample.csv',encoding=encoding)

In [ ]:
df.head()

### Creating SQLite Database

In [7]:
# Create a connection object
sqliteConnection = sqlite3.connect('sales.db')

In [ ]:
# Convert the dataframe to an SQL table
df.to_sql('Sales', sqliteConnection, if_exists='replace')

### Importing Langchain modules

In [9]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.sql_database import SQLDatabase

In [10]:
# Create an instance of SQLDatabase
database = SQLDatabase.from_uri('sqlite:///sales.db')

### Creating Agent

In [11]:

agent_excuter = create_sql_agent(
    llm=model,
    toolkit=SQLDatabaseToolkit(db=database,llm=model),
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors = True,
    verbose=True,
    return_intermediate_steps=True
)

In [ ]:
query = "What will be the total sales for January 2003?"

agent_excuter.run(query)

# New ChatBot

### Hybird Search Langchain

In [1]:
%pip install pinecone-client pinecone-text pinecone-notebooks

Note: you may need to restart the kernel to use updated packages.


In [2]:
pinecone_API_KEY = "pcsk_2QZwDt_EsA9kwmK4azJGavUqToMCibU1LCpsgwAb3ebvvMhmoEFrcd7d9wZ9EELgw9CJDZ"

In [3]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [4]:
import os
from pinecone import Pinecone,ServerlessSpec

index_name = "hybrid-search"
pc = Pinecone(api_key=pinecone_API_KEY)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384, #Dimension of Dense Vector
        metric="dotproduct",  # sparse value supported only for DotProduct
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
        )

In [5]:
index = pc.Index(index_name)
index

In [6]:
## Vector Embedding and Sparse Matrix
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [7]:
# Sparse Matrix
from pinecone_text.sparse import BM25Encoder
bm25encoder = BM25Encoder().default()
bm25encoder

In [8]:
import nltk
nltk.download('punkt_tab')

sentences =[
        "In 2023, the total sales for January was 1000",
        "In 2022, the total sales for February was 700",
        "In 2021, the total sales for March was 500",
]

bm25encoder.fit(sentences)
bm25encoder.dump("bm25_values.json")
bm25encoder=BM25Encoder().load("bm25_values.json")

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\eDominer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


  0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
retriever = PineconeHybridSearchRetriever(embeddings=embeddings, index=index, sparse_encoder=bm25encoder)
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x0000018B3F327E00>, index=<pinecone.data.index.Index object at 0x0000018B12C971A0>)

In [10]:


retriever.add_texts(
    [
        "In 2023, the total sales for January was 1000",
        "In 2022, the total sales for February was 700",
        "In 2021, the total sales for March was 500",
    ]
)

  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
retriever.invoke("total sales in January 2023")

PydanticUserError: If you use `@root_validator` with pre=False (the default) you MUST specify `skip_on_failure=True`. Note that `@root_validator` is deprecated and should be replaced with `@model_validator`.

For further information visit https://errors.pydantic.dev/2.10/u/root-validator-pre-skip